In [1]:
# импортируем необходимые библиотеки, классы и функции
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# увеличиваем количество строк вывода
pd.set_option('display.max_rows', 400)

In [2]:
# загружаем данные
data = pd.read_csv('Data/santander_train.csv')

In [3]:
# удаляем ID
data.drop('ID', axis=1, inplace=True)

In [4]:
# разбиваем набор данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('TARGET', axis=1),
    data['TARGET'],
    test_size=0.3,
    random_state=42)

# смотрим количество наблюдений и признаков
print(X_train.shape, X_test.shape)

(53214, 369) (22806, 369)


In [5]:
# импортируем необходимые классы и функции 
# для нашей функции importance_auc()
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# пишем функцию, вычисляющую AUC для модели с одним признаком
def importance_auc(train, y_train, imp_strategy):
    # задаем количество десятичных разрядов
    pd.set_option('display.precision', 3)
    # создаем копию обучающего набора
    train_copy = train.copy()
    # создаем список признаков
    col_list = train_copy.select_dtypes(include=['number']).columns
    # создаем пустой список, в который будем записывать значения AUC
    auc_list = []
    # вычисляем AUC для модели с одним признаком,
    # используя перекрестную проверку
    for i in col_list:
        pipe = Pipeline([('imputer', SimpleImputer(strategy=imp_strategy)),
                         ('scaler', StandardScaler()), 
                         ('logreg', LogisticRegression(solver='liblinear'))])     
        auc = cross_val_score(pipe, train_copy[[i]], y_train, 
                              scoring='roc_auc', cv=5).mean()
        # добавляем значение AUC в список
        auc_list.append(auc)
    # превращаем список со значениями AUC в серию
    auc_values = pd.Series(auc_list)
    # метками индекса будут имена признаков
    auc_values.index = col_list
    # сортируем по убыванию
    auc_values = auc_values.sort_values(ascending=False)
    return auc_values

In [6]:
# применяем функцию, вычисляя значения AUC
auc_values = importance_auc(X_train, y_train, 'median')
auc_values

var15                            0.698
saldo_var30                      0.697
num_meses_var5_ult3              0.694
num_var30                        0.682
saldo_var42                      0.682
saldo_var5                       0.677
saldo_medio_var5_hace2           0.677
saldo_medio_var5_ult3            0.676
saldo_medio_var5_ult1            0.676
ind_var30                        0.675
saldo_medio_var5_hace3           0.672
num_var42                        0.672
ind_var5                         0.667
num_var5                         0.667
num_var35                        0.648
num_var4                         0.648
var36                            0.619
var38                            0.585
num_var30_0                      0.537
ind_var39_0                      0.531
ind_var41_0                      0.530
num_var39_0                      0.530
num_var41_0                      0.530
ind_var12_0                      0.527
num_var12_0                      0.527
num_var13_0              

In [7]:
# отбираем столбцы с AUC выше 0.5
select_columns = auc_values.index[auc_values > 0.5].tolist()
# вычисляем количество отобранных столбцов
len(select_columns)

233